#### 一. 惰性删除
1. 哈希表往往采用闭散列策略处理冲突.  
2. 闭散列的策略通常为为线性试探,所以相同hash值的key会集中在哈希表的某一段.  
 由于这种局部性, 导致在删除节点时产生空桶二破坏这种局部性. 因此可谓每个桶设置一个标记, 指示该桶虽然目前为空, 但此前曾存放过词条

In [1]:
class HashTable(object):
    def __init__(self,bucket_number=5):
        '''bucket_number:哈希表长,也是桶的数量
           entry_number:词条数量'''
        self.bucket_number = bucket_number
        self.entry_number = 0 #词条数量
        self.ht = [] # 桶数组
        self.lazy_removal = [0 for i in range(bucket_number)] # 懒删除标记